In [ ]:
!pip install openai google-cloud-bigquery

In [ ]:
!pip install pybigquery


In [ ]:
import os
import re
import pandas as pd
from google.cloud import bigquery
from openai import OpenAI

class GoogleSmartQueryAgent:
    def __init__(self, project_id, dataset_id, openai_api_key):
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.client = bigquery.Client(project=project_id)
        self.openai = OpenAI(api_key= openai_api_key)
        self.schema = self._load_schema()

    def _load_schema(self):
        schema = {}
        dataset_ref = self.client.dataset(self.dataset_id)
        for table in self.client.list_tables(dataset_ref):
            table_id = table.table_id
            full_table_ref = f"{self.project_id}.{self.dataset_id}.{table_id}"
            table_obj = self.client.get_table(full_table_ref)
            schema[table_id] = {
                "columns": [field.name for field in table_obj.schema],
                "types": {field.name: field.field_type for field in table_obj.schema},
                "row_count": table_obj.num_rows
            }
        return schema

    def _select_relevant_table(self, question):
        scores = {}
        q_words = set(re.findall(r'\w+', question.lower()))
        for table, info in self.schema.items():
            relevance = 0
            if any(word in table.lower() for word in q_words):
                relevance += 10
            for col in info["columns"]:
                if any(word in col.lower() for word in q_words):
                    relevance += 2
            scores[table] = relevance
        return max(scores, key=scores.get)

    def _generate_sql(self, question: str, table: str):
        # Prompt to OpenAI to generate SQL
        prompt = f"""
        You're an expert SQL assistant working with BigQuery.
        The table `{table}` has the following columns: {', '.join(self.schema[table]['columns'])}.

        Write a valid SQL query to answer: "{question}"

        When dealing with date filters in natural language, apply these replacements:
        - "today" → CURRENT_DATE()
        - "yesterday" → DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
        - "past 7 days" or "last 7 days" or "last week" → BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE()

        If comparing or filtering against a TIMESTAMP column like `snapshot_date`, always cast it using `DATE(snapshot_date)` before comparing with date expressions.

        Use the full table reference: `{self.project_id}.{self.dataset_id}.{table}`.
        Limit results to 10 rows unless more are explicitly required.

        Only return the SQL code. Do not explain anything.
        """


        completion = self.openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return completion.choices[0].message.content.strip()

    def ask(self, question: str):
      """Ask a question and return clean final answer from BigQuery"""
      try:
          # price trends
          if any(word in question.lower() for word in ["rising", "falling", "trend", "going up", "going down"]):
              table = "price_trend_flags"
          else:
              table = self._select_relevant_table(question)

          raw_sql = self._generate_sql(question, table)

          #cleaning SQL
          cleaned_sql = re.sub(r"^```sql|```", "", raw_sql, flags=re.MULTILINE).strip()
          cleaned_sql = re.sub(r"(?i)^Sure.*?\n", "", cleaned_sql).strip()

          df = self.client.query(cleaned_sql).to_dataframe()

          if df.empty:
              return "No results found."

          return df.to_string(index=False)

      except Exception as e:
          return f"Error: {e}"

if __name__ == "__main__":
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "bq_key.json"  

    PROJECT_ID = "n8nauth-465703"
    DATASET_ID = "dbt_pbhatia_crypto_dataset"

    agent = GoogleSmartQueryAgent(PROJECT_ID, DATASET_ID, OPENAI_API_KEY)

